In [1]:
%pip install Sastrawi

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle
from joblib import dump
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('stopwords')

In [ ]:
data = pd.read_csv('data/dataset-september.csv', delimiter=';')
df_stopwords = pd.read_csv('data/stopwords-id.csv', header=None, names=['stopword'])
df_slang = pd.read_csv('data/kamus-singkatan.csv', delimiter=';', names=['singkatan', 'kata'])
df_lexicon = pd.read_csv('data/lexicon-inset.csv')
lexicon_dict = dict(zip(df_lexicon['word'], df_lexicon['weight']))
df_corpus = pd.read_csv('data/corpus.csv')
corpus_words = df_corpus['kata'].tolist()

stemmer = StemmerFactory().create_stemmer()

In [ ]:
def cleaning(full_text): 
    full_text = full_text.replace('-ness', ' ')
    full_text = full_text.replace('-jualness', ' ')
    full_text = re.sub(r'@\S+', ' ', full_text)
    full_text = re.sub(r'^RT[\s]+', ' ', full_text)
    full_text = re.sub(r'[^a-zA-Z]', ' ', full_text)
    full_text = re.sub(r'/n', ' ', full_text)
    full_text = re.sub(r'http\S+', ' ', full_text)
    full_text = re.sub(r'https\S+', ' ', full_text)
    full_text = re.sub(r'www\S+', ' ', full_text)
    full_text = re.sub(r'\d+', ' ', full_text)
    full_text = re.sub(r'(?<!\bunnes)(\w)(\1+)(?=\s|[\.,!])', ' ', full_text)
    full_text = full_text.strip(' ')
    full_text = full_text.lower()
    
    tokens = nltk.word_tokenize(full_text)
    
    punc = list(punctuation)
    words = [w for w in tokens if w not in punc]
    stop_words = set(stopwords.words('indonesian'))
    words = [w.lower() for w in words if not w.lower() in stop_words]

    # lemmatization
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemming_tokens = [stemmer.stem(token) for token in words]

    text = ' '.join(stemming_tokens)
    
    return text

In [ ]:
def replace_word(text):
    elongated_words = re.findall(r'\b\w*(?:(\w)\1{2,})\w*\b', text)
    for word in elongated_words:
        replacement = word[0]
        text = re.sub(r'\b' + re.escape(word) + r'\b', replacement, text)
    return text

In [ ]:
def translate_slang(text_list):
    translated_list = []
    for text in text_list:
        words = text.split()
        translated_words = []
        for word in words:
            if word in df_slang['singkatan'].tolist():
                translated_words.append(df_slang[df_slang['singkatan'] == word]['kata'].values[0])
            else:
                translated_words.append(word)
        translated_list.append(' '.join(translated_words))
    return translated_list

In [ ]:
def remove_stopwords(tokens):
    if isinstance(tokens, list):
        filtered_words = [word for word in tokens if word.lower() not in df_stopwords['stopword'].values]
        return filtered_words
    else:
        return tokens
    
def stemming(tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemming_tokens = [stemmer.stem(token) for token in tokens]
    stemming_words = ' '.join(stemming_tokens)
    return stemming_words

In [ ]:
def labelling(text):
  tokens = text.split()
  score = 0
  
  for token in tokens:
    if token in lexicon_dict:
      score += lexicon_dict[token]

  polarity = ''
  if score > 0:
    polarity = 'Positif'
  elif score < 0:
    polarity = 'Negatif'
  else:
    polarity = 'Netral'
  
  return polarity

In [ ]:
def tokenize(text):
    text = word_tokenize(text)
    return text

In [ ]:
# def proses(data):
#     data['clean_text'] = data['full_text'].apply(cleaning)
#     data['clean_text'] = data['clean_text'].apply(replace_word)
#     data['clean_text'] = data['clean_text'].apply(tokenize)
#     data['clean_text'] = data['clean_text'].apply(translate_slang)
#     data['clean_text'] = data['clean_text'].apply(remove_stopwords)
#     data['clean_text'] = data['clean_text'].apply(stemming)
#     data['sentimen'] = data['clean_text'].apply(labelling)
#     return data


In [ ]:
data['clean_text'] = data['full_text'].apply(cleaning)
data

In [ ]:
data['clean_text'] = data['clean_text'].apply(replace_word)
data

In [ ]:
data['clean_text'] = data['clean_text'].apply(tokenize)
data

In [ ]:
data['clean_text'] = data['clean_text'].apply(translate_slang)
data

In [ ]:
data['clean_text'] = data['clean_text'].apply(remove_stopwords)
data

In [ ]:
data['clean_text'] = data['clean_text'].apply(stemming)
data

In [ ]:
data = pd.read_excel(r'D:\10.media_social_analysis-main\After_Processed_Dataset_PASAL (2).xlsx')
data

In [ ]:
data['sentimen'] = data['full_text'].apply(labelling)
data

In [ ]:
# if __name__ == '__main__':
#     proses(data)

In [ ]:
data = pd.read_excel(r'C:\Users\Ahmad Dani\codingz\SENTIS\app\hasil_sentimen.xlsx')
data

In [ ]:
data = pd.read_excel(r'C:\Users\Ahmad Dani\codingz\SENTIS\app\hasil_sentimen.xlsx')
data

In [ ]:
data = pd.read_excel(r'C:\Users\Ahmad Dani\codingz\SENTIS\app\hasil_sentimen.xlsx')

In [ ]:
data.dropna(subset=['clean_text', 'sentimen'], inplace=True)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier

# Memisahkan fitur (X) dan label (y) dari dataset
X = data['clean_text']
y = data['sentimen']

# Penggunaan TF-IDF untuk vektorisasi teks
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X)
X_test_tfidf = vectorizer.transform(X)  # Transformasi data uji

In [ ]:
from imblearn.over_sampling import SMOTE
# Ganti Random Over Sampling dengan SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_train_tfidf, y)

# Buat DataFrame baru
os = pd.DataFrame(list(zip([x[0] for x in X_resampled], y_resampled)), columns=['full_text', 'Label'])

# Tampilkan distribusi kelas setelah penerapan SMOTE
os['Label'].value_counts()

In [ ]:
# Pembagian data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, stratify=y_resampled, test_size=0.1, random_state=42)

In [ ]:
# # Inisialisasi dan pelatihan model SVM dengan kernel radial basis function (RBF)
# svm_model = SVC(kernel='rbf', C=100.0, gamma=0.03)
# svm_model.fit(X_train_tfidf, y_train)
estimators = [
    ('lr', LogisticRegression()),        # Estimator 1: Logistic Regression
    ('rf', RandomForestClassifier()),    # Estimator 2: Random Forest Classifier
    ('dt', DecisionTreeClassifier()),    # Estimator 3: Decision Tree Classifier
    ('svm', SVC(kernel='rbf', C=1.0, gamma=0.01))    # Estimator 5: SVM Classifier
]

# Definisikan StackingClassifier dengan meta-estimator LogisticRegression
svm_model = StackingClassifier(
    estimators=estimators,               # Daftar estimator yang akan digunakan
    final_estimator=RandomForestClassifier()# Estimator meta untuk menggabungkan hasil estimator lainnya
)

svm_model.fit(X_train, y_train)

# Evaluasi akurasi model pada data uji
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluasi model dengan classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Simpan model dan objek vectorizer untuk penggunaan selanjutnya
dump(vectorizer, 'models/vectorizer1.pkl')
dump(svm_model, 'models/ensemble_model4.pkl')


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Membuat confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix dengan label 'Negatif', 'Netral', 'Positif'
labels = ['Negatif', 'Netral', 'Positif']
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
%pip install flask